In [20]:
%idle_timeout 2880
%glue_version 4.0
%worker_type G.1X
%number_of_workers 2

import sys
from awsglue.dynamicframe import DynamicFrame
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
import pyspark.pandas as pd
from pyspark.sql.types import *
from pyspark.sql.functions import col, lpad, to_timestamp

sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

You are already connected to a glueetl session 6ea8cfde-9f3f-4bfd-9c8e-ed99e3fd7394.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Current idle_timeout is 2880 minutes.
idle_timeout has been set to 2880 minutes.


You are already connected to a glueetl session 6ea8cfde-9f3f-4bfd-9c8e-ed99e3fd7394.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Setting Glue version to: 4.0


You are already connected to a glueetl session 6ea8cfde-9f3f-4bfd-9c8e-ed99e3fd7394.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Previous worker type: G.1X
Setting new worker type to: G.1X


You are already connected to a glueetl session 6ea8cfde-9f3f-4bfd-9c8e-ed99e3fd7394.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Previous number of workers: 2
Setting new number of workers to: 2



###Create a DynamicFrame from a table in the AWS Glue Data Catalog and display its schema


In [21]:
bronze_olist_customers_dataset = glueContext.create_dynamic_frame.from_catalog(database='ecommerce-datalake', table_name='olistcustomersdataset')
bronze_olist_geolocation_dataset = glueContext.create_dynamic_frame.from_catalog(database='ecommerce-datalake', table_name='olistgeolocationdataset')
bronze_olist_orderitems_dataset = glueContext.create_dynamic_frame.from_catalog(database='ecommerce-datalake', table_name='olistorderitemsdataset')
bronze_olist_orderpayments_dataset = glueContext.create_dynamic_frame.from_catalog(database='ecommerce-datalake', table_name='olistorderpaymentsdataset')
bronze_olist_orderreviews_dataset = glueContext.create_dynamic_frame.from_catalog(database='ecommerce-datalake', table_name='olistorderreviewsdataset')
bronze_olist_orders_dataset = glueContext.create_dynamic_frame.from_catalog(database='ecommerce-datalake', table_name='olistordersdataset')
bronze_olist_products_dataset = glueContext.create_dynamic_frame.from_catalog(database='ecommerce-datalake', table_name='olistproductsdataset')
bronze_olist_sellers_dataset = glueContext.create_dynamic_frame.from_catalog(database='ecommerce-datalake', table_name='olistsellersdataset')
bronze_product_category_name_translation = glueContext.create_dynamic_frame.from_catalog(database='ecommerce-datalake', table_name='productcategorynametranslation')

In [22]:
#Customer df cleanup
df_bronze_olist_customers_dataset = bronze_olist_customers_dataset.toDF()
df_bronze_olist_customers_dataset = df_bronze_olist_customers_dataset.dropDuplicates()
df_bronze_olist_customers_dataset = df_bronze_olist_customers_dataset.withColumn('customer_zip_code_prefix', col('customer_zip_code_prefix').cast('string'))
df_bronze_olist_customers_dataset = df_bronze_olist_customers_dataset.withColumn('customer_zip_code_prefix', lpad(col('customer_zip_code_prefix'), 5, '0'))
df_bronze_olist_customers_dataset = df_bronze_olist_customers_dataset.withColumnRenamed('customer_zip_code_prefix', 'customer_zip_code')
df_bronze_olist_customers_dataset_cleaned = DynamicFrame.fromDF(df_bronze_olist_customers_dataset, glueContext)

In [34]:
datasink1 = glueContext.write_dynamic_frame.from_options(frame = df_bronze_olist_customers_dataset_cleaned, connection_type = "s3", connection_options = {"path": "s3://ecommerce-silver/customers/"}, format = "parquet")


In [23]:
#Seller df cleanup
df_bronze_olist_sellers_dataset = bronze_olist_sellers_dataset.toDF()
df_bronze_olist_sellers_dataset = df_bronze_olist_sellers_dataset.dropDuplicates()
df_bronze_olist_sellers_dataset = df_bronze_olist_sellers_dataset.withColumn('seller_zip_code_prefix', col('seller_zip_code_prefix').cast('string'))
df_bronze_olist_sellers_dataset = df_bronze_olist_sellers_dataset.withColumn('seller_zip_code_prefix', lpad(col('seller_zip_code_prefix'), 5, '0'))
df_bronze_olist_sellers_dataset = df_bronze_olist_sellers_dataset.withColumnRenamed('seller_zip_code_prefix', 'seller_zip_code')
df_bronze_olist_sellers_dataset_cleaned = DynamicFrame.fromDF(df_bronze_olist_sellers_dataset, glueContext)

In [35]:
datasink2 = glueContext.write_dynamic_frame.from_options(frame = df_bronze_olist_sellers_dataset_cleaned, connection_type = "s3", connection_options = {"path": "s3://ecommerce-silver/sellers/"}, format = "parquet")


In [24]:
#Product category lookup df cleanup
df_bronze_product_category_name_translation = bronze_product_category_name_translation.toDF()
df = df_bronze_product_category_name_translation
header = df_bronze_product_category_name_translation.filter((df['col0'] == 'product_category_name') & (df_bronze_product_category_name_translation['col1'] == 'product_category_name_english'))
restDF = df_bronze_product_category_name_translation.subtract(header)
headerColumn = header.first()
for column in restDF.columns:
    restDF = restDF.withColumnRenamed(column, headerColumn[column])
df_bronze_product_category_name_translation_cleaned = DynamicFrame.fromDF(restDF, glueContext)

In [26]:
#Product df cleanup
df_bronze_olist_products_dataset = bronze_olist_products_dataset.toDF()
df_bronze_olist_products_dataset = df_bronze_olist_products_dataset.dropDuplicates()
df_bronze_olist_products_dataset = df_bronze_olist_products_dataset \
    .join(restDF,
          on='product_category_name', 
          how='left_outer')
df_bronze_olist_products_dataset = df_bronze_olist_products_dataset \
    .withColumn('product_category_name_english',
                col('product_category_name_english')
                .cast(StringType())) \
    .fillna({'product_category_name': 'Uncategorized',
             'product_category_name_english': 'Uncategorized'})
df_bronze_olist_products_dataset_cleaned = DynamicFrame.fromDF(df_bronze_olist_products_dataset, glueContext)

In [36]:
datasink3 = glueContext.write_dynamic_frame.from_options(frame = df_bronze_olist_products_dataset_cleaned, connection_type = "s3", connection_options = {"path": "s3://ecommerce-silver/product/"}, format = "parquet")


In [27]:
#Order df cleanup
df_bronze_olist_orders_dataset = bronze_olist_orders_dataset.toDF().toPandas()

new_header = df_bronze_olist_orders_dataset.iloc[0] 
df_bronze_olist_orders_dataset = df_bronze_olist_orders_dataset[1:] 
df_bronze_olist_orders_dataset.columns = new_header
df_bronze_olist_orders_dataset = spark.createDataFrame(df_bronze_olist_orders_dataset)
df_bronze_olist_orders_dataset = df_bronze_olist_orders_dataset.dropDuplicates()

df_bronze_olist_orders_dataset = df_bronze_olist_orders_dataset \
    .withColumn('order_purchase_timestamp', to_timestamp(col('order_purchase_timestamp'), 'yyyy-MM-dd HH:mm:ss')) \
    .withColumn('order_approved_at', to_timestamp(col('order_approved_at'), 'yyyy-MM-dd HH:mm:ss')) \
    .withColumn('order_delivered_carrier_date', to_timestamp(col('order_delivered_carrier_date'), 'yyyy-MM-dd HH:mm:ss')) \
    .withColumn('order_delivered_customer_date', to_timestamp(col('order_delivered_customer_date'), 'yyyy-MM-dd HH:mm:ss')) \
    .withColumn('order_estimated_delivery_date', to_timestamp(col('order_estimated_delivery_date'), 'yyyy-MM-dd HH:mm:ss'))

df_bronze_olist_orders_dataset_cleaned = DynamicFrame.fromDF(df_bronze_olist_orders_dataset, glueContext)

/opt/amazon/spark/python/lib/pyspark.zip/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/opt/amazon/spark/python/lib/pyspark.zip/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


In [37]:
datasink4 = glueContext.write_dynamic_frame.from_options(frame = df_bronze_olist_orders_dataset_cleaned, connection_type = "s3", connection_options = {"path": "s3://ecommerce-silver/orders/"}, format = "parquet")


In [28]:
#Order items df cleanup
df_bronze_olist_orderitems_dataset = bronze_olist_orderitems_dataset.toDF()
df_bronze_olist_orderitems_dataset = df_bronze_olist_orderitems_dataset.drop_duplicates()
date_format = 'yyyy-MM-dd HH:mm:ss'
df_bronze_olist_orderitems_dataset = df_bronze_olist_orderitems_dataset \
    .withColumn('shipping_limit_date', to_timestamp(col('shipping_limit_date'), date_format))
df_bronze_olist_orderitems_dataset_cleaned = DynamicFrame.fromDF(df_bronze_olist_orderitems_dataset, glueContext)

/opt/amazon/spark/python/lib/pyspark.zip/pyspark/sql/dataframe.py:127: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


In [38]:
datasink5 = glueContext.write_dynamic_frame.from_options(frame = df_bronze_olist_orderitems_dataset_cleaned, connection_type = "s3", connection_options = {"path": "s3://ecommerce-silver/orders_items/"}, format = "parquet")


In [29]:
#Order payments df cleanup
df_bronze_olist_orderpayments_dataset = bronze_olist_orderpayments_dataset.toDF()
df_bronze_olist_orderpayments_dataset = df_bronze_olist_orderpayments_dataset.drop_duplicates()
df_bronze_olist_orderpayments_dataset_cleaned = DynamicFrame.fromDF(df_bronze_olist_orderpayments_dataset, glueContext)

In [39]:
datasink6 = glueContext.write_dynamic_frame.from_options(frame = df_bronze_olist_orderpayments_dataset_cleaned, connection_type = "s3", connection_options = {"path": "s3://ecommerce-silver/orders_payments/"}, format = "parquet")


In [30]:
#Order review df cleanup
df_bronze_olist_orderreviews_dataset = bronze_olist_orderreviews_dataset.toDF()
df_bronze_olist_orderreviews_dataset = df_bronze_olist_orderreviews_dataset.drop_duplicates()
date_format = 'yyyy-MM-dd HH:mm:ss'
df_bronze_olist_orderreviews_dataset = df_bronze_olist_orderreviews_dataset \
    .withColumn('review_creation_date', to_timestamp(col('review_creation_date'), date_format)) \
    .withColumn('review_answer_timestamp', to_timestamp(col('review_answer_timestamp'), date_format))
df_bronze_olist_orderreviews_dataset_cleaned = DynamicFrame.fromDF(df_bronze_olist_orderreviews_dataset, glueContext)

In [40]:
datasink7 = glueContext.write_dynamic_frame.from_options(frame = df_bronze_olist_orderreviews_dataset_cleaned, connection_type = "s3", connection_options = {"path": "s3://ecommerce-silver/orders_reviews/"}, format = "parquet")


In [31]:
#Order geolocation df cleanup
df_bronze_olist_geolocation_dataset = bronze_olist_geolocation_dataset.toDF()
df_bronze_olist_geolocation_dataset = df_bronze_olist_geolocation_dataset.drop_duplicates()
df_bronze_olist_geolocation_dataset_cleaned = DynamicFrame.fromDF(df_bronze_olist_geolocation_dataset, glueContext)